In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

First I explore the dataset for just Philadelphia. Below I move to the whole dataset and answer the questions that were posed.


In [2]:
philly = pd.read_csv('../input/philadelphia_9-24-2016_9-30-2017.csv')

philly.head()

In [3]:
philly.describe()

In [4]:
philly.fillna('O')

In [14]:
types = philly[['Variety','Date', 'Package', 'High Price', 'Origin', 'Item Size' ]]

types.head()

In [ ]:
types['Variety'].describe()

In [ ]:
types.dtypes

In [ ]:
types_group = types.groupby('Variety')
types_group.size()

In [ ]:
types_totals = types_group.sum()


In [ ]:
variety_plot = types_totals.plot(kind='bar')

Howden type is the most popular kind of pumpkin sold in Philadelphia.


Where are most of the pumpkins in Philadelphia from?

In [ ]:
origin_group = types.groupby('Origin')
origin_totals = origin_group.sum()

origin_plot = origin_totals.plot(kind='bar')

PA that makes sense. Next lets see when people in Philly are buying pumpkins

9/23 is the most common week to buy pumpins in Philadelphia. What is the connetion between size of pumpkin and price??

In [ ]:
size_group = types.groupby('Item Size')
size_totals = size_group.head()

size_plot = size_totals.plot(kind='bar')

In [25]:
kind_group = types.groupby(['Date', 'Item Size']).sum()
kind_group.head()

In [26]:
kind_group.unstack().head()

In [28]:
my_plot = kind_group.unstack().plot(kind='bar', stacked=True, title='Pumpkin by Date and price')
my_plot.set_xlabel("Date Purchased")
my_plot.set_ylabel("Money spent")
pylab.legend(loc=9, bbox_to_anchor=(0.5, -0.5))


In the above graph we can see the size of the pumpkin by date purchased. Extra large pumpkins were exclusively bought on 9/23.

**Now that we've looked at the Philly data, lets take this country-wide.**

In [3]:
import glob
glob.glob("../input/*.csv")

In [4]:
all_data = pd.DataFrame()

for f in glob.glob("../input/*.csv"):
    df = pd.read_csv(f)
    all_data = all_data.append(df,ignore_index=True)

In [5]:
all_data.describe()

In [6]:
extract_data = all_data[['Variety','Date', 'Package', 'High Price', 'Origin', 'Item Size', 'City Name' ]]

extract_data.describe()

First I am going to make the package into 24 or 36 so we have a numerical understanding of how many pumpkins are in that bin

In [9]:
cleanup_bins = {"Package": {"24 inch bins":24, "36 inch bins":36}}

In [10]:
extract_data.replace(cleanup_bins, inplace=True)
extract_data.head()


Lets do the same to item size

In [11]:
cleanup_items = {"Item Size": {"xlge":4, "lge":3, "med-lge":2.5, "med":2, "sml":1}}
extract_data.replace(cleanup_items, inplace=True)
extract_data.head()


In [12]:
extract_data.describe()

In [13]:
extract_data.dtypes

In [14]:
extract_data = extract_data.convert_objects(convert_numeric=True)



In [15]:
extract_data.dtypes

In [16]:
extract_data.describe()

**Which state produces the most pumpkins?**

It's difficult to determine this exactly from the dataset, given there's no obivous way to determine the number of pumpkins in each bin. But below I have compiled the number of bins of 24" and 36" and displayed them with their state of origin. I think it will give a good approximate answer.



In [67]:
state_data = extract_data[['Origin', 'Package', 'Item Size']]

state_group = state_data.groupby(['Origin', 'Package']).sum()

my_plot = state_group.unstack().plot(kind='bar', stacked=True, title='States and Pumpkin Production')
my_plot.set_xlabel("Origin")
my_plot.set_ylabel("Quantity")
pylab.legend(loc=9, bbox_to_anchor=(0.5, -0.5))


It appears that PA grows the most pumpkins.

**Where are pumpkins priced the highest?**

-Here I will use the average high price broken down by city.

In [18]:
price_data = extract_data[['City Name', 'High Price']]

price_group = price_data.groupby(['City Name']).mean()

my_plot = price_group.unstack().plot(kind='bar', stacked=True, title='City and Prices of Pumpkins')
my_plot.set_xlabel("City Name")
my_plot.set_ylabel("Price of Pumpkins")


Boston has the most expsensive pumpkins.

**How does pumpkin size relate to pumpkin price?******

In [23]:

size_data = all_data[['Item Size', 'High Price']]
size_group = size_data.groupby(['Item Size']).mean()

my_plot = size_group.unstack().plot(kind='bar', stacked=True, title='Pumpkin size and Price')
my_plot.set_xlabel("Pumpkin sizes")
my_plot.set_ylabel("Price of Pumpkins")




It appears that the most expsenive pumpkins are extra large, and the least expensive pumpkins are small. 

**Which pumpkin type costs the most? the least?**

In [26]:
size_data = extract_data[['Variety', 'High Price']]
size_group = size_data.groupby(['Variety']).mean()

my_plot = size_group.unstack().plot(kind='bar', stacked=True, title='Pumpkin type and Price')
my_plot.set_xlabel("Pumpkin Types")
my_plot.set_ylabel("Price of Pumpkins")


Blue type is the most expensive. The least expensive is miniature (no surprise on that).